In [1]:
import cv2
import mediapipe as mp
import os
import csv
import math
import numpy as np

# ---- Helper functions ----
def calculate_angle(a, b, c):
    ba = (a[0]-b[0], a[1]-b[1])
    bc = (c[0]-b[0], c[1]-b[1])
    dot_product = ba[0]*bc[0] + ba[1]*bc[1]
    mag_ba = math.sqrt(ba[0]**2 + ba[1]**2)
    mag_bc = math.sqrt(bc[0]**2 + bc[1]**2)
    cosine_angle = dot_product / (mag_ba * mag_bc + 1e-6)
    return math.degrees(math.acos(max(min(cosine_angle, 1), -1)))

def calculate_distance(a, b):
    return math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

def flip_image(image):
    return cv2.flip(image, 1)

def rotate_image(image, angle):
    h, w = image.shape[:2]
    matrix = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
    return cv2.warpAffine(image, matrix, (w, h))

def adjust_brightness(image, factor):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv = np.array(hsv, dtype=np.float64)
    hsv[:,:,2] = hsv[:,:,2]*factor
    hsv[:,:,2][hsv[:,:,2]>255]  = 255
    hsv = np.array(hsv, dtype=np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# ---- Pose setup ----
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5)

# ---- CSV output ----
csv_file = open('yoga_expanded_dataset.csv', mode='w', newline='')
csv_writer = csv.writer(csv_file)

header = [
    # Elbows and shoulders
    "left_shoulder_angle", "right_shoulder_angle",
    "left_elbow_angle", "right_elbow_angle",
    # Hips and knees
    "left_hip_angle", "right_hip_angle",
    "left_knee_angle", "right_knee_angle",
    # Distances
    "shoulder_distance", "hip_distance",
    "l_shoulder_to_hip", "r_shoulder_to_hip",
    "l_shoulder_to_ankle", "r_shoulder_to_ankle",
    "label"
]
csv_writer.writerow(header)

# ---- Dataset path ----
dataset_path = "TRAIN"

# ---- Process images + augmentations ----
for pose_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, pose_name)
    if not os.path.isdir(folder_path):
        continue

    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        image = cv2.imread(img_path)
        if image is None:
            continue

        # All variations to process: original + augmentations
        variations = [
            image,
            flip_image(image),
            rotate_image(image, 10),
            rotate_image(image, -10),
            adjust_brightness(image, 1.2),
            adjust_brightness(image, 0.8)
        ]

        for aug_img in variations:
            image_rgb = cv2.cvtColor(aug_img, cv2.COLOR_BGR2RGB)
            results = pose.process(image_rgb)

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark
                landmarks = [(lm[i].x, lm[i].y) for i in range(33)]

                # Angles at joints
                left_shoulder_angle = calculate_angle(landmarks[13], landmarks[11], landmarks[23])
                right_shoulder_angle = calculate_angle(landmarks[14], landmarks[12], landmarks[24])
                left_elbow_angle = calculate_angle(landmarks[11], landmarks[13], landmarks[15])
                right_elbow_angle = calculate_angle(landmarks[12], landmarks[14], landmarks[16])
                left_hip_angle = calculate_angle(landmarks[11], landmarks[23], landmarks[25])
                right_hip_angle = calculate_angle(landmarks[12], landmarks[24], landmarks[26])
                left_knee_angle = calculate_angle(landmarks[23], landmarks[25], landmarks[27])
                right_knee_angle = calculate_angle(landmarks[24], landmarks[26], landmarks[28])

                # Distances
                shoulder_distance = calculate_distance(landmarks[11], landmarks[12])
                hip_distance = calculate_distance(landmarks[23], landmarks[24])
                l_shoulder_to_hip = calculate_distance(landmarks[11], landmarks[23])
                r_shoulder_to_hip = calculate_distance(landmarks[12], landmarks[24])
                l_shoulder_to_ankle = calculate_distance(landmarks[11], landmarks[27])
                r_shoulder_to_ankle = calculate_distance(landmarks[12], landmarks[28])

                # Write row
                csv_writer.writerow([
                    left_shoulder_angle, right_shoulder_angle,
                    left_elbow_angle, right_elbow_angle,
                    left_hip_angle, right_hip_angle,
                    left_knee_angle, right_knee_angle,
                    shoulder_distance, hip_distance,
                    l_shoulder_to_hip, r_shoulder_to_hip,
                    l_shoulder_to_ankle, r_shoulder_to_ankle,
                    pose_name
                ])

csv_file.close()
print("Expanded feature dataset saved to yoga_expanded_dataset.csv")


KeyboardInterrupt: 